## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:35,  8.02it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:36<00:00, 203.41it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Matthew Hocter,Australia,Writer.,2020-06-01 12:58:50,11,75,370,False,2020-08-04 07:57:03,Hearing of friends in Brazil losing family to ...,[COVID19],Twitter Web App,False
0,WEF Energy,Geneva,Tweets from the World Economic Forum on energy...,2012-11-07 17:08:47,2575,839,1370,False,2020-08-04 07:57:01,This powerful tool can help drive Europe's gre...,None,Buffer,False
0,Carles Dijous (AAlb),"Barcelona, Spain",Colaborador docente UOC (Universitat Oberta de...,2008-12-28 11:08:16,8968,5534,701,False,2020-08-04 07:57:00,Science Round-Up • #Coronavirus experts: #Seco...,"[Coronavirus, SecondWave, Covid19]",TweetDeck,False
0,Rajprasong_News,"Bangkok, Thailand","Aka Sean Boonpracong: Pol analyst, fmr Thai Tr...",2010-01-22 15:23:08,12546,966,8591,False,2020-08-04 07:56:55,#Vietnam #Covid19 spreads to 10 different plac...,"[Vietnam, Covid19, Hanoi, HoChiMinhCity]",Twitter for iPhone,False
0,Premier Lubabalo Oscar Mabuyane,"Bhisho, EC South Africa",Official Twitter of Premier Lubabalo Oscar Mab...,2018-10-02 18:03:01,10239,478,654,False,2020-08-04 07:56:52,"Today, we’re in #NelsonMandelaBay visiting all...",[NelsonMandelaBay],Twitter for iPhone,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (66936, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 66936 all tweets: 74436


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (74436, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)